In [ ]:
import pandas as pd
from IPython.display import display

# Filtragem de dados
### Carregamento de dados 

In [ ]:
# Carregar o dataset
df = pd.read_csv("tabela analise.csv")

In [ ]:
# Carregar o dataset do notion
df_notion = pd.read_csv("tabela_analise_notion.csv")

## Remoção de linhas não utilizáveis

In [ ]:
# Remoção das linhas não utilizadas
df = df[df["Meio"] != "CASH OUT"]

# Resetar o índice após a remoção
df = df.reset_index(drop=True)

In [ ]:
# Remoção das linhas não utilizadas
df = df[~((df["Tokens"] >= 500) & (df["Meio"] == "Gift"))]

In [ ]:
df_notion = df_notion.drop(columns=["Entradas", "Obs:", "Mês", "Ano","Dia"])

In [ ]:
df_notion["Data"] = pd.to_datetime(df_notion["Data"], format="%d/%m/%Y")  # Garante o formato correto
df_notion = df_notion[df_notion["Data"].dt.year != 2025]  # Mantém apenas anos diferentes de 2025


## Conversão de dados para valores numéricos e de data

In [ ]:
# Converte a linha de Tokens de string para numeric
df["Tokens"] = pd.to_numeric(df["Tokens"], errors="coerce")  # Converte para número, ignorando erros

In [ ]:
df["Data e hora"] = pd.to_datetime(df["Data e hora"], format="%m/%d/%Y %H:%M:%S")

# Separando a data e a hora
df["data"] = df["Data e hora"].dt.strftime("%d/%m/%Y")  # Converte para DD/MM/YYYY
df["hora"] = df["Data e hora"].dt.strftime("%H:%M:%S")  # Apenas a hora

# Removendo a coluna original
df = df.drop(columns=["Data e hora"])


In [ ]:
df_notion["Data"] = pd.to_datetime(df_notion["Data"], format="%m/%d/%Y %H:%M:%S")

# Separando a data
df_notion["Data"] = df_notion["Data"].dt.strftime("%d/%m/%Y")  # Converte para DD/MM/YYYY excluindo informações de horário


In [ ]:
#checando resultados
display (df.head(10))

## Padronização de valores com uma formatação mais clara

In [ ]:
#visualização de valores únicos para a padronização
df_viz = pd.read_excel("dados_formatados.xlsx")

In [ ]:
valores_unicos = df_viz["Meio"].unique()

print(valores_unicos)

print(f"Valores Unicos totais: {valores_unicos.size}")

In [ ]:
df_viz["Meio"] = df_viz["Meio"].apply(lambda x: "Tip menu" if any(c in x for c in ['request']) else x)

In [ ]:
# Reordenando as colunas para que "data" seja a primeira e "hora" a segunda
colunas_ordenadas = ["data", "hora"] + [col for col in df.columns if col not in ["data", "hora"]]
df = df[colunas_ordenadas]

In [ ]:
df = df.sort_values(by="data", ascending=True)

In [ ]:
df_viz.to_excel("dados_formatados.xlsx", index=False)

In [ ]:
colunas_ordenadas = ["Data"] + [col for col in df_notion.columns if col not in ["Data"]]
df_notion = df_notion[colunas_ordenadas]

In [ ]:
#checando resultados
display(df_notion.head(10))

In [ ]:
df_notion["Horas trabalhadas"] = df_notion["Horas"]

# Removendo a coluna original
df_notion = df_notion.drop(columns=["Horas"])

#Adicionando uma nova coluna para tratamento posterior
df_notion["Horas trabalhadas by number"] = df_notion["Horas trabalhadas"]

In [ ]:
#visualização de valores únicos para a padronização
valores_unicos = df_notion["Minutos_trabalhados"].unique()

display (valores_unicos)

display (f"Valores Unicos totais: {valores_unicos.size}")

In [ ]:
renomear = {
    'Não trabalhei': '0',
    'Folga        ': '0',
    '2hrs'         : '120',
    '3hrs'         : '180',
    '4hrs'         : '240',
    '5hrs'         : '300',
    '6hrs'         : '360',
    '7hrs'         : '420',
    '8hrs'         : '480',
    '1hr, 30min'   : '90',
    '2hrs, 30min'  : '150',
    '30min, 3hrs'  : '210',
    '30min, 4hrs'  : '270',
    '30min, 5hrs'  : '330',
    '30min, 6hrs'  : '390',
    '30min, 7hrs'  : '450',
}


df_notion["Horas trabalhadas by number"] = df_notion["Horas trabalhadas by number"].replace(renomear)

#df_notion["Horas trabalhadas by number"] = df_notion["Horas trabalhadas by number"].apply(lambda x: "0" if any(c in x for c in ["Não trabalhei", "Folga"]) else x)

In [ ]:
df_notion = df_notion.dropna(subset=["Horas trabalhadas by number"])

In [ ]:
df_notion["Horas trabalhadas by number"] = pd.to_numeric(df_notion["Horas trabalhadas by number"], errors="coerce")

In [ ]:
df_notion["Minutos_trabalhados"] = df_notion["Horas trabalhadas by number"]

# Removendo a coluna original
df_notion = df_notion.drop(columns=["Horas trabalhadas by number"])

In [ ]:
df["hora"] = pd.to_datetime(df["hora"], format="%H:%M:%S").dt.hour

# Criando os intervalos e os rótulos
bins = [0, 6, 12, 18, 24]  # Intervalos de horas
labels = ["Madrugada", "Manhã", "Tarde", "Noite"]  # Rótulos

# Aplicando a categorização de forma vetorizada
df["Periodo"] = pd.cut(df["hora"], bins=bins, labels=labels, right=False)

## Lançamento de dados em outro DB

In [ ]:
df.to_excel("dados_formatados.xlsx", index=False)

In [ ]:
df_notion.to_excel("dados_formatados_notion.xlsx", index=False)